Take a look at the [repository](https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix) for more information

# Install

In [47]:
import os

os.chdir('/')
!rm -r /CycleGAN-VAE-NST/
!git clone https://github.com/davexhardware/CycleGAN-VAE-NST

Cloning into 'CycleGAN-VAE-NST'...
remote: Enumerating objects: 2598, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 2598 (delta 42), reused 49 (delta 18), pack-reused 2515 (from 1)
Receiving objects: 100% (2598/2598), 8.28 MiB | 17.16 MiB/s, done.
Resolving deltas: 100% (1616/1616), done.


In [48]:
import os
os.chdir('CycleGAN-VAE-NST/')

In [39]:
!pip install -r requirements.txt

# Datasets

Download one of the official datasets with:

-   `bash ./datasets/download_cyclegan_dataset.sh [apple2orange, summer2winter_yosemite, horse2zebra, monet2photo, cezanne2photo, ukiyoe2photo, vangogh2photo, maps, cityscapes, facades, iphone2dslr_flower, ae_photos]`

Or use your own dataset by creating the appropriate folders and adding in the images.

-   Create a dataset folder under `/dataset` for your dataset.
-   Create subfolders `testA`, `testB`, `trainA`, and `trainB` under your dataset's folder. Place any images you want to transform from a to b (cat2dog) in the `testA` folder, images you want to transform from b to a (dog2cat) in the `testB` folder, and do the same for the `trainA` and `trainB` folders.

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [53]:
import zipfile
import glob
os.chdir('/')
!unzip -d /trainB -o /content/drive/MyDrive/trainB_pt_16_128.zip
!unzip -d /trainA /content/drive/MyDrive/trainA_pt_16_128.zip

File_paths = glob.glob('/trainA')
Fp= glob.glob('/trainB')

Output streaming troncato alle ultime 5000 righe.
  inflating: /trainA/trainA_pt_16_128/063226.jpg.pt  
  inflating: /trainA/trainA_pt_16_128/063237.jpg.pt  
  inflating: /trainA/trainA_pt_16_128/063240.jpg.pt  
  inflating: /trainA/trainA_pt_16_128/063260.jpg.pt  
  inflating: /trainA/trainA_pt_16_128/063265.jpg.pt  
  inflating: /trainA/trainA_pt_16_128/063267.jpg.pt  
  inflating: /trainA/trainA_pt_16_128/063301.jpg.pt  
  inflating: /trainA/trainA_pt_16_128/063305.jpg.pt  
  inflating: /trainA/trainA_pt_16_128/063306.jpg.pt  
  inflating: /trainA/trainA_pt_16_128/063323.jpg.pt  
  inflating: /trainA/trainA_pt_16_128/063328.jpg.pt  
  inflating: /trainA/trainA_pt_16_128/063342.jpg.pt  
  inflating: /trainA/trainA_pt_16_128/063344.jpg.pt  
  inflating: /trainA/trainA_pt_16_128/063375.jpg.pt  
  inflating: /trainA/trainA_pt_16_128/063395.jpg.pt  
  inflating: /trainA/trainA_pt_16_128/063405.jpg.pt  
  inflating: /trainA/trainA_pt_16_128/063425.jpg.pt  
  inflating: /trainA/trainA_pt_1

In [54]:
!cd /
!mv /trainA/trainA_pt_16_128/ /CycleGAN-VAE-NST/datasets/trainA
import random
path='/CycleGAN-VAE-NST/datasets/trainA'
if os.path.exists(path):
  images = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
  random.shuffle(images)
  images = images[:3000]
  for image in [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]:
    if image not in images:
        os.remove(os.path.join(path, image))
path='/CycleGAN-VAE-NST/datasets/trainB'
if os.path.exists(path):

  for file in [f for f in os.listdir(path)]:
    os.remove(os.path.join(path,file))
  os.rmdir(path)
!cd /
!mv /trainB/trainB_pt_16_128/ /CycleGAN-VAE-NST/datasets/trainB

# Training

-   `python train.py --dataroot ./datasets/horse2zebra --name horse2zebra --model cycle_gan`

Change the `--dataroot` and `--name` to your own dataset's path and model's name. Use `--gpu_ids 0,1,..` to train on multiple GPUs and `--batch_size` to change the batch size. I've found that a batch size of 16 fits onto 4 V100s and can finish training an epoch in ~90s.

Once your model has trained, copy over the last checkpoint to a format that the testing model can automatically detect:

Use `cp ./checkpoints/horse2zebra/latest_net_G_A.pth ./checkpoints/horse2zebra/latest_net_G.pth` if you want to transform images from class A to class B and `cp ./checkpoints/horse2zebra/latest_net_G_B.pth ./checkpoints/horse2zebra/latest_net_G.pth` if you want to transform images from class B to class A.


In [20]:

import torch

print(torch.cuda.is_available())

True


In [ ]:
os.chdir('/CycleGAN-VAE-NST/')
!python train.py --dataroot ./datasets --name portraits2op --lambda_identity 0 --latent_dim 30 --ngf 32 --ndf 32 --norm batch --netG resnet_6blocks --lambdaB 8.0 --dataset_mode tensor --crop_size 128 --batch_size 8 --verbose --n_epochs 20 --n_epochs_decay 30 --display_id -1

----------------- Options ---------------
               batch_size: 8                             	[default: 1]
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                crop_size: 128                           	[default: 256]
                 dataroot: ./datasets                    	[default: None]
             dataset_mode: tensor                        	[default: unaligned]
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: -1                            	[default: 1]
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                           
          

# Testing

-   `python test.py --dataroot datasets/horse2zebra/testA --name horse2zebra_pretrained --model test --no_dropout`

Change the `--dataroot` and `--name` to be consistent with your trained model's configuration.

> from https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix:
> The option --model test is used for generating results of CycleGAN only for one side. This option will automatically set --dataset_mode single, which only loads the images from one set. On the contrary, using --model cycle_gan requires loading and generating results in both directions, which is sometimes unnecessary. The results will be saved at ./results/. Use --results_dir {directory_path_to_save_result} to specify the results directory.

> For your own experiments, you might want to specify --netG, --norm, --no_dropout to match the generator architecture of the trained model.

In [ ]:
!python test.py --dataroot datasets/horse2zebra/testA --name horse2zebra_pretrained --model test --no_dropout

# Visualize

In [ ]:
import matplotlib.pyplot as plt

img = plt.imread('./results/horse2zebra_pretrained/test_latest/images/n02381460_1010_fake.png')
plt.imshow(img)

In [ ]:
import matplotlib.pyplot as plt

img = plt.imread('./results/horse2zebra_pretrained/test_latest/images/n02381460_1010_real.png')
plt.imshow(img)